<a href="https://colab.research.google.com/github/Val2425/MachineLearningProject-Korea2024/blob/main/MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. Import Data**

In [ ]:
#Import dataset from Kaggle
from google.colab import files

# Upload Kaggle API key file (kaggle.json)
uploaded = files.upload()
del uploaded

Saving kaggle.json to kaggle.json


In [ ]:
# Configure Kaggle API credentials
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset using Kaggle API
import kagglehub
path = kagglehub.dataset_download("bhavikjikadara/fake-news-detection")
print("Path to dataset files:", path)

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Path to dataset files: /root/.cache/kagglehub/datasets/bhavikjikadara/fake-news-detection/versions/1


In [ ]:
cd /root/.cache/kagglehub/datasets/bhavikjikadara/fake-news-detection/versions/1

/root/.cache/kagglehub/datasets/bhavikjikadara/fake-news-detection/versions/1


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load true and fake news datasets
true_df = pd.read_csv('true.csv')
fake_df = pd.read_csv('fake.csv')

In [ ]:
# Replacing abbreviation to have a uniform format for dates
mois = {
    'Jan ': 'January ',
    'Feb ': 'February ',
    'Mar ': 'March ',
    'Apr ': 'April ',
    'May ': 'May ',
    'Jun ': 'June ',
    'Jul ': 'July ',
    'Aug ': 'August ',
    'Sep ': 'September ',
    'Oct ': 'October ',
    'Nov ': 'November ',
    'Dec ': 'December '
}
fake_df['date'] = fake_df['date'].replace(mois, regex=True)

# Converting dates to Date variables
true_df['date'] = pd.to_datetime(true_df['date'], errors='coerce')
fake_df['date'] = pd.to_datetime(fake_df['date'], errors='coerce')

print(true_df['date'].sample(10))
print(fake_df['date'].sample(10))

9532    2016-05-16
18214   2017-10-05
12889   2017-12-07
3800    2017-05-12
20543   2017-09-09
13776   2017-11-27
20290   2017-09-11
8761    2016-07-13
10717   2016-02-23
19885   2017-09-16
Name: date, dtype: datetime64[ns]
12132   2016-12-15
73      2017-11-22
1032    2017-06-23
15007   2015-10-29
6079    2016-06-01
5486    2016-07-13
7024    2016-04-08
6361    2016-05-15
3810    2016-11-14
4058    2016-10-24
Name: date, dtype: datetime64[ns]


In [ ]:
# Add a label column: 1 for true news, 0 for fake news
true_df['label'] = 1
fake_df['label'] = 0

# Combine both datasets into a single DataFrame
df = pd.concat([true_df, fake_df], ignore_index=True)

#**2. Date column**

#**3. Subject column**

In [ ]:
# Standardize the 'subject' column across datasets
subject_mapping = {
    'News': 'General News',
    'US_News': 'General News',
    'worldnews': 'General News',
    'politics': 'Politics',
    'politicsNews': 'Politics',
    'left-news': 'Politics',
    'Middle-east': 'General News',
    'Government News': 'Politics'
}
df['subject'] = df['subject'].map(subject_mapping)

# Drop the 'subject' column because it is not relevant for the analysis
df = df.drop(columns=['subject'])

#**4. Feature engineering**

In [ ]:
# Function to calculate uppercase letter percentage in text with rounding to 2 decimal places
def calculate_uppercase_proportion(text):
    if len(text) == 0:
        return 0
    uppercase_count = sum(1 for char in text if char.isupper())
    percentage = (uppercase_count / len(text))
    return round(percentage, 3)  # Round to two decimal places

# Add a column for uppercase percentage in titles
df['uppercase_proportion'] = df['title'].apply(calculate_uppercase_proportion)

#**5. Title and Text columns**

In [ ]:
# Delete all text until "(Reuters) - " in the dataset
df['text'] = df['text'].str.replace(r'^.*\(Reuters\) - ', '', regex=True)

# Convert text to lowercase and strip whitespace
df['title'] = df['title'].str.lower().str.strip()
df['text'] = df['text'].str.lower().str.strip()

# Ensure no NaN values in 'title' and 'text' before removing special characters
df['title'] = df['title'].fillna('').str.replace(r'[^a-zA-Z0-9\s]', ' ', regex=True)
df['text'] = df['text'].fillna('').str.replace(r'[^a-zA-Z0-9\s]', ' ', regex=True)

# Ensure no multiple spaces in the text and rejoin into sentences
df['title'] = df['title'].str.split().str.join(' ')
df['text'] = df['text'].str.split().str.join(' ')

# Remove duplicates from the combined DataFrame
df.drop_duplicates(keep='first', inplace=True)

# Remove rows where 'title' or 'text' are empty strings
df = df[(df['title'] != '') & (df['text'] != '')]

In [ ]:
print(df.sample(20))

                                                   title  \
37042  federally funded riot baltimore mayor wants fe...   
43487  nothing big mac donald trump jr caught in late...   
36165  will hillary s secret weapon be a game changer...   
35448  the truth about why hillary is the only candid...   
30956  pam geller rips into geraldo for insults about...   
6598   mistrial declared in corruption case against e...   
6722   trump s tough trade talk makes u s firms fear ...   
37623  bam commie scammer mustafa ali the epa s envir...   
198    senator grassley expresses reservations on two...   
27740  noam chomsky warns americans the gop is the mo...   
3116   senate s mcconnell draft healthcare bill expec...   
20808  after year of repression in bahrain west remai...   
29374  trump dominates south carolina dooming the rep...   
43437  syria north korea trump s axis of evil is bigg...   
2395   trump administration reaches deal with texas c...   
10068  pentagon chief wants streamlined 